In [1]:
# 크롤링 작업을 위한 라이브러리 임포트
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import requests
import pandas as pd
import json
import time   # 코드 진행 지연을 위한 time 임포트
from selenium.webdriver.common.by import By   # 2022년 7월 이후 selenium 업데이트로 인한 xpath 추적 임포트
from selenium.webdriver.common.keys import Keys

In [2]:
##### 카카오 주소를 좌표(위도, 경도)로 변환 api
## 개인 api key를 할당 받아 적용
api_key = '054a38bc8bc9ee2f8fadaaad44e27f97'
def addr_to_lat_lon(addr):
#     print(addr)
    url = f'https://dapi.kakao.com/v2/local/search/address.json?query={addr}'
#     print(url)
    headers = {"Authorization": "KakaoAK " + api_key}
    result = json.loads(str(requests.get(url, headers=headers).text))

    match_first = result['documents'][0]['address']
    return float(match_first['x']), float(match_first['y'])

In [6]:
!pip install folium

  Using cached folium-0.13.0-py2.py3-none-any.whl (96 kB)


In [3]:
##### 카카오맵 minimap으로 생성
## No module named 'folium' 라는 에러가 뜬다면
## anaconda cmd 창에서 "pip install folium" 으로 설치
import folium
from folium.plugins import MiniMap

def make_map(df, region):
    # 해당 지역 시/도청 좌표 Search
    yp = addr_to_lat_lon(region)[0]
    xp = addr_to_lat_lon(region)[1]
    
    # 지도 생성하기
    m = folium.Map(location=[xp, yp],   # 뽑은 좌표
                   zoom_start=12)
    # 미니맵 추가하기
    minimap = MiniMap() 
    m.add_child(minimap)
    # 마커 추가하기
    for i in range(len(df.index)):
        folium.Marker(location=[df['위도'].iloc[i],df['경도'].iloc[i]],
                  popup=df['점수'].iloc[i],
                  tooltip=[df['식당명'].iloc[i], "\n점수", df['점수'].iloc[i]]
                  ).add_to(m)
    return m

In [4]:
region = str(input("검색을 원하는 지역의 \'시, 도\'를 입력해 주세요 : "))
page   = int(input("검색 범위(페이지)를 입력해주세요. : "))
starpoint = input("최대점수를 입력해 주세요(4.5 만점) : ")

검색을 원하는 지역의 '시, 도'를 입력해 주세요 : 서울시
검색 범위(페이지)를 입력해주세요. : 6
최대점수를 입력해 주세요(4.5 만점) : 3.7


In [5]:
driver = webdriver.Chrome("chromedriver")
#### time.sleep() 을 주기적으로 넣어 서버 측에 부하를 막고, 사람으로 인식하도록 처리한다
time.sleep(1)
driver.get('https://map.kakao.com/')
time.sleep(2)

#### 입력 받은 region에 ' 식당\n'이라는 string 을 추가해 날린다.
#### 마지막에 '\n'을 추가한 이유는 'enter'와 같은 기능을 한다고 보면 된다(검색어 입력)
driver.find_element(By.XPATH, '//*[@id="search.keyword.query"]').send_keys('%s 식당\n' % region)
time.sleep(2)

#### 검색 완료 후, '정확도 순'에 맞춰 식당 데이터들을 정렬한다.
driver.find_element(By.XPATH, '//*[@id="info.search.place.sort"]/li[1]/a').send_keys(Keys.ENTER)
time.sleep(2)

In [6]:
n_list = []   # 식당 이름 리스트
s_list = []   # 식당 별점 리스트
a_list = []   # 식당 주소 리스트
px_list = []  # 경도 리스트
py_list = []  # 위도 리스트
ka_dict = {}  # 별점에 따른 가게를 담을 딕셔너리

In [7]:
def parsing_data(n_list, s_list, a_list, px_list, py_list):
    source = driver.page_source
    parsed_source = bs(source, 'html.parser')

    names_div_list = parsed_source.find_all("li",class_ = 'PlaceItem clickArea')        # 가게명
    names_span_list = parsed_source.find_all('span', class_ = 'screen_out')
    span_names = names_span_list[1:31:2]
    for i in range(len(span_names)):
        names = span_names[i].text
        n_list.append(names)

    div_scores = parsed_source.find_all('em', class_ = "num") 
    del div_scores[-1]
    for i in range(len(div_scores)):
        scores = div_scores[i].text
        s_list.append(scores)

    div_address = parsed_source.find_all('div', class_ = "addr")                        # 주소
    for i in range(len(div_address)):
        address = div_address[i].text.replace('\n','').split('(지번)')[0]
        try:
            px_list.append(addr_to_lat_lon(address)[0])
            py_list.append(addr_to_lat_lon(address)[1])
        except:
            print(f"{i+1}번 가게 주소 데이터 error")
        a_list.append(address)
    time.sleep(2)

In [8]:
## 페이지 5보다 작을 때
if page < 5:
    for j in range(1,page+1):
        parsing_data(n_list, s_list, a_list, px_list, py_list)
            
        driver.find_element(By.XPATH,'//*[@id="info.search.page.no%s"]' % (j+1)).send_keys(Keys.ENTER)        # 페이지 넘기기
    print(f"{page}번쨰 페이지 입력")
## 각 묶음 페이지의 1쪽 해당
for j in range(page//5):
    parsing_data(n_list, s_list, a_list, px_list, py_list)
    
## 각 묶음 페이지의 2, 3, 4, 5쪽 해당
    for i in range(2,7):
        time.sleep(1)
        if i == 6:
            if page%5 == 0:
                continue
            driver.find_element(By.XPATH,'//*[@id="info.search.page.next"]').send_keys(Keys.ENTER)
            time.sleep(2)
        else:
            driver.find_element(By.XPATH,'//*[@id="info.search.page.no%s"]' % i).send_keys(Keys.ENTER)
            time.sleep(2)

        parsing_data(n_list, s_list, a_list, px_list, py_list)
        
## 입력한 페이지가 속한 묶음 페이지에 해당
if page > 5:
    for k in range(2,page%5+1):

        driver.find_element(By.XPATH,'//*[@id="info.search.page.no%s"]' % k).send_keys(Keys.ENTER)
        time.sleep(2)

        parsing_data(n_list, s_list, a_list, px_list, py_list)

## 데이터 딕셔너리 형태로 적재
for i in range(len(n_list)):
#    if i%5 == 0:
#        print(f"========{i//5}페이지========")
    try:
        ka_dict[i] = (n_list[i], s_list[i], a_list[i],py_list[i],px_list[i])
    except:
        print(f"{n_list[i]} : 식당 데이터 오류")

for i in range(len(ka_dict)):
    if ka_dict[i][1] < starpoint:
        del ka_dict[i]
kakao = pd.DataFrame(ka_dict).T
kakao.columns=['식당명', '점수', '주소', '위도', '경도']
make_map(kakao, region)

2번 가게 주소 데이터 error
15번 가게 주소 데이터 error
2번 가게 주소 데이터 error
울프강스테이크하우스 : 식당 데이터 오류
농민백암왕순대 : 식당 데이터 오류
어글리베이커리 : 식당 데이터 오류


In [9]:
print(f"조회된 식당 갯수 : {len(kakao.index)}개")
kakao

조회된 식당 갯수 : 39개


,식당명,점수,주소,위도,경도
2,금돼지식당,3.8,서울 중구 다산로 149,37.557092,127.011667
3,중앙해장,3.9,서울 강남구 영동대로86길 17 육인빌딩 1층,37.508283,127.06548
6,노티드 청담,3.7,서울 강남구 도산대로53길 15,37.524192,127.038242
13,삼육가 사당역1호점,4.2,서울 서초구 방배천로 10 1층,37.477393,126.983099
22,다운타우너 안국,4.1,서울 종로구 북촌로 6-4 1층,37.577468,126.986372
24,진작,3.7,서울 중구 수표로12길 12 1층,37.564018,126.990837
25,핏제리아오,4.6,"서울 종로구 동숭길 86 2, 3층",37.582096,127.004326
26,우래옥 본점,3.9,서울 중구 창경궁로 62-29,37.568219,126.998714
31,어반플랜트 합정,4.2,서울 마포구 독막로4길 3 1~3층,37.547569,127.04245
32,제스티살룬 성수,3.9,서울 성동구 서울숲4길 13 1층,37.559522,127.005088


# 추후 작업

### 코로플레스
 => 지도 상에 밀집도에 따라 색상 입력

# 추가 정보

- plotly 라이브러리
- 상관관계 히트맵
- 미리캔버스(https://www.miricanvas.com/)